In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab01.ipynb")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
import re
import itertools
from functools import reduce
import math

In [ ]:
alphabet = "abdcedfghijklmnopqrstuvwxyz"

# Lab 1: Substitution Ciphers
Contributions From: Ryan Cottone

Welcome to Lab 1! In this lab, we will explore the most common type of early cipher -- the substitution cipher.
By the end of lab, you will have built a full encrypting and decrypting system complete with a cracking function.

Please review the slides and Note 1 if you are unsure what a substitution cipher is.

## Helpers

In [ ]:
def char_to_num(text):
    return ord(text.lower()) - 97;

In [ ]:
def num_to_char(num):
    return chr(num + 97).lower();

In [ ]:
def shift_letter(char, shift):
    if not char.isalpha():
        return char
    else:
        return num_to_char((char_to_num(char) + shift) % 26) 

In [ ]:
def clean_text(text):
    return re.sub('[^A-Za-z\s!?.,$]*', '', text.lower())

**Question 1**: Complete the following implementation of a Caesar Cipher.

In [ ]:
def caesar_encrypt(plaintext, shift):
    plaintext = clean_text(plaintext)
    encrypted = "" 
    for i in range(len(plaintext)):
        ...
    return encrypted

In [ ]:
def caesar_decrypt(ciphertext, shift):
    ciphertext = clean_text(ciphertext)
    decrypted = "" 
    for i in range(len(ciphertext)):
        ...
    return decrypted

In [ ]:
grader.check("q1")

**Question 2.1**: Complete the following implementation of key padding.

In [ ]:
def pad_key(key, length):
    key = clean_text(key)
    newkey = key
    for i in range(len(key), length):
        ...
    return newkey

In [ ]:
grader.check("q2_1")

**Question 2.2**: Complete the following implementation of a Vignére cipher.

In [ ]:
def vignere_encrypt(plaintext, key):
    plaintext = clean_text(plaintext)
    padded_key = ...
    encrypted = ""
    
    for i in range(len(plaintext)):
        ...
    
    return encrypted

In [ ]:
def vignere_decrypt(ciphertext, key):
    ciphertext = clean_text(ciphertext)
    padded_key = ...
    decrypted = ""
    
    for i in range(len(ciphertext)):
        ...
        
    return decrypted

In [ ]:
grader.check("q2_2")

**Question 3.1**: Complete the following implementation of a frequency analysis tool.

In [ ]:
def count_letters(text):
    # Return a dictionary with the counts of every letter.
    counts = {}
    text = text.lower()
    
    for letter in alphabet:
        counts[letter] = 0;
    
    ...
    
    return counts;

In [ ]:
grader.check("q3_1")

In [ ]:
def calculate_proportions(text): # Coded for you
    counts = count_letters(text).values()
    nparr = np.fromiter(counts, dtype=float)
    return nparr / sum(counts)

In [ ]:
def plot_freqs_from_text(text): # Coded for you
    plot_freqs(calculate_proportions(text))

In [ ]:
def plot_freqs(freqs): # Coded for you
    plt.bar([alphabet[i] for i in range(26)], freqs)

Now, let's try with a long text:

In [ ]:
sample_text = ""
with open('sample.txt', 'r') as file:
    sample_text = file.read().replace('\n', '')

In [ ]:
plot_freqs_from_text(sample_text)

In [ ]:
english_frequencies = {
    'a': .0812,
    'b': .0149,
    'c': .0271,
    'd': .0432,
    'e': .1202,
    'f': .0230,
    'g': .0203,
    'h': .0592,
    'i': .0731,
    'j': .0100,
    'k': .0069,
    'l': .0398,
    'm': .0261,
    'n': .0695,
    'o': .0768,
    'p': .0182,
    'q': .0011,
    'r': .0602,
    's': .0628,
    't': .0910,
    'u': .0288,
    'v': .0111,
    'w': .0209,
    'x': .0017,
    'y': .0211,
    'z': .0007
}
np_english_frequencies = np.fromiter(english_frequencies.values(), dtype=float)

In [ ]:
plot_freqs(english_frequencies.values())

In [ ]:
def plot_overlay(freq1, freq2): # Coded for you
    plt.bar([alphabet[i] for i in range(26)],freq1, color='orange', width = 0.5)
    plt.bar([alphabet[i] for i in range(26)], freq2, color='blue', alpha=0.5)

In [ ]:
plot_overlay(np_english_frequencies, calculate_proportions(sample_text))

As you can see, the frequencies of longer texts are almost identical to the frequency of English as a whole. 

**Question 3.2**: Complete the following implementation of Total Variation Distance.

In [ ]:
def tvd(freq1, freq2):
    # Takes in two numpy arrays of length 26.
    ...

In [ ]:
grader.check("q3_2")

In [ ]:
tvd(np_english_frequencies, calculate_proportions(sample_text))

Our text has a relatively low TVD with the base English frequencies. Remember, the lower the TVD = the closer the two distributions are. Let's see how the TVD of an encrypted version of the sample text compares to English.

In [ ]:
encrypted_sample_text = caesar_encrypt(sample_text, 8)
tvd(np_english_frequencies, calculate_proportions(encrypted_sample_text))

That is a lot higher! If we chart these two, you will see why it is so large:

In [ ]:
plot_overlay(np_english_frequencies, calculate_proportions(encrypted_sample_text))

You can see that the histograms are completely different. In fact, you can make out the shift of the blue to the right from the orange.

**Question 3.3**: Build a function to minimize the TVD over all 26 possible shifts.

In [ ]:
def find_best_shift(ciphertext):
    ...

The following test may take a few seconds to run.

In [ ]:
grader.check("q3_3")

**Question 3.4**: Build a function to decrypt an arbitrary Caesar ciphertext.

In [ ]:
def crack_caesar(ciphertext):
    shift = ...
    ...

Try it out for yourself! Longer texts will be decrypted much more reliably than shorter ones (can you see why?).

In [ ]:
codebreaking_message = "Hello from Codebreaking at Cal!"
shift = 5 # Change this if you want!
encrypted_codebreaking_message = caesar_encrypt(codebreaking_message, shift)
print("Encrypted ciphertext:", encrypted_codebreaking_message)
print("Cracked plaintext:", crack_caesar(encrypted_codebreaking_message))

In [ ]:
grader.check("q3_4")

Unfortunately, a simple frequency analysis will not work for polyalphabetic ciphers like Vignére. For example, let's see what happens when we plot the frequencies of our sample text encoded with the key "samplekey":

In [ ]:
plot_overlay(np_english_frequencies, calculate_proportions(vignere_encrypt(sample_text, "sample key")))

As you can see, our frequency chart lines up very poorly with overall english frequencies -- there is no noticeable shift. If we try to run our find_best_shift function, we will see that it returns nothing more than gibberish.

In [ ]:
shift = find_best_shift(vignere_encrypt(sample_text, "sample key"))
attempted_decryption = caesar_decrypt(sample_text, shift)
plot_overlay(np_english_frequencies, calculate_proportions(attempted_decryption))
print(attempted_decryption[:50] + '...')

In order to remedy this, we use a key principle that applies to polyalphabetic ciphers -- they are simply a combination of *monoalphabetic* ciphers. If we can split the polyalphabetic cipher into numerous monoalphabetic ones, we can solve those individually and re-combine to produce our decrypted text. For example, take the Vignere ciphertext "bcbc" with key "ab". 'a' would be the key for the first and third letter, and 'b' is the key for the second and forth letter. Thus, we can split this into two Caesar ciphers -- "bb" and "cc". We find that our keys are 'b' and 'c', respectively, then we concatenate to find the overall key "bc". 

The tricky part, however, is finding the key length. We can't know how to split the cipher unless we know how many ciphers to split into. Fortunately, there exists a very useful tool called the *Kasiski test* to determine the key length of a polyalphabetic substitution cipher. Simply put, we look for repeated substrings of words and check the distance between the two. For example: 

"Cryptography isnt cryptocurrency"
"abcd"
"**Csastp**iuaqjb itpw **csastp**exrsgqcz"

Notice the two bolded substrings, which are exactly 16 characters apart (start-start). This would imply our key could be 16 characters long or any of the factors of 16 (8, 4, 2), since the key would "repeat" in time for it to encrypt the same plaintext to the same ciphertext. It could be any of these, but over a long text, only the key lengths and its multiples (4, 8, 12 ..) will show up in appreciable amounts. (A notable exception is 2, just because its really small and a lot of repeats happen by chance).

Our algorithm for determining the key length is as follows:

1. Find all repeated substrings of length 3 or 4 and their distances. Put this data into a dictionary with (length -> count)
2. Compare the proportions of these values to those of an unencrypted, very long text (same idea as comparing to base English frequency values)
3. Collect those which are significantly above the base values and determine the key length based on least-inversions principle (this part is a bit tricky and subjective, so it is implemented for you)
4. Try the best *k* keylengths and compare using a frequency analysis on the final product (k is set by the user).


In [ ]:
def find_repeated_substrings(text):
    
    text=clean_text(text)
        
    appearances = defaultdict(lambda: 0)
    
    for i in range(2,len(text)//2):
        
        counts = defaultdict(lambda:-1)
        
        for k in range(0, len(text), i):
            if (k+3 > len(text)):
                continue
                        
            snippet_three = text[k:k+3]
            snippet_four = text[k:k+4]


            if (counts[snippet_three] != -1):
                appearances[i] += 1;
            
            if (counts[snippet_four] != -1 and k+4 < len(text)):
                appearances[i] += 1;
            
            counts[snippet_three] = k
            counts[snippet_four] = k
    
    return appearances

MAX_KEYLENGTH defines the maximum length of key to check for. Theoretically, this could be up to len(text) - 1, but for practical purposes we will set it at 20. Feel free to change and see how the program works!

In [ ]:
MAX_KEYLENGTH = 20

In [ ]:
def keylen_proportions_from_text(text):
    parsed = np.fromiter(find_repeated_substrings(text).values(), dtype=int)[:MAX_KEYLENGTH*2]
    return parsed/sum(parsed)

In [ ]:
def approx(x):
    offset = 0.012
    if (x == 2):
        offset+=0.02
    elif (x > 2 and x < 6):
        offset-=  0.01
    return 0.45* 0.7**x + offset

In [ ]:
base_keylen_proportions = keylen_proportions_from_text(sample_text)

In [ ]:
colors = []
for i in range(2, MAX_KEYLENGTH+3):
    if (i % 5 == 0):
        colors.append('#50514f')
    elif (i % 4 == 0):
        colors.append('#f25f5c')
    elif (i % 3 == 0):
        colors.append('#ffae4a')
    elif (i % 2 == 0):
        colors.append('#247ba0')
    else:
        colors.append('#70c1b3')
        
fig, ax = plt.subplots()

ax.bar(range(2,MAX_KEYLENGTH+3), base_keylen_proportions[:MAX_KEYLENGTH+1], color=colors)
ax.set_xticks(range(2,MAX_KEYLENGTH+3))
ax.plot(range(2,MAX_KEYLENGTH+3), [approx(x) for x in range(2,MAX_KEYLENGTH+3)] ,color="black")
plt.show()

**Question 4.1**: Implement find_diff_from_base, which finds the differences of a given ciphertext from the base keylength proportions.

In [ ]:
def find_diff_from_base(text):
    text_keylen_proportions = ...
        
    diffs = text_keylen_proportions - base_keylen_proportions[:len(text_keylen_proportions)]
    
    return diffs    

In [ ]:
grader.check("q4_1")

Now that we have find_diff_from_base, let's visualize what diffs of various keylengths look like when plotted.

In [ ]:
EXAMPLE_KEYLENGTH = 5 # Change between 2 and 20!

diffs = find_diff_from_base(vignere_encrypt(sample_text, alphabet[0:EXAMPLE_KEYLENGTH]))
fig, ax = plt.subplots()
ax.bar(range(2,MAX_KEYLENGTH+3), diffs[:MAX_KEYLENGTH+1], color=colors)
ax.set_xticks(range(2,MAX_KEYLENGTH+3))
plt.show()

You'll notice that the key length and its multiples are noticeably above 0 while the rest tend to be lower. (2 is an often exception, its so small that a lot of statistical noise gets in the way).

Detecting the best key length programmatically is often challenging, and has a few different approaches. The one below scores different lengths based on whether any given multiple is less than its previous multiple, as well as how many in between are larger than the given multiple. As you see in the example graph, each successive multiple should be less than the previous. If you try a key length like 4, you'll see 2 < 4 and 6 < 8, which helps disqualify 2 as a potential key length.

In [ ]:
def find_best_divisor(nums, diffs): 
    result = []
    k = 1
    for num in nums:
        score = 0
        for i in range(2*num, min(MAX_KEYLENGTH*2+1, len(diffs)), num):
            inbetween = sum([diffs[a-2] > diffs[i-2] for a in range(i-num+1, i)]) / len(range(i-num+1, i))
            
            if inbetween > 0.1:
                score -= 1
            if (diffs[i-num-2] < diffs[i - 2]):
                score -= 1
        k+=1
        
        result.append((num, score))
    result = sorted(result, key=lambda x: x[1], reverse=True)
    top_three = [x[0] for x in result[:3]] 

    return top_three + [reduce(math.gcd, top_three)]

**Question 4.2:** Implement find_vignere_key_lengths, which takes in a ciphertext and returns the best three keylengths.

*HINT: Use find_diff_from_base and find_best_divisor!*

In [ ]:
def find_vignere_key_lengths(ciphertext):
    diffs = ...
    
    if len(diffs) == 0:
        return [2,3,4] # Handles edge cases for low-length texts
    
    potential = []
    for i in range(2, len(diffs)+2):
        if diffs[i-2] > 0:
            ...
    ...

In [ ]:
grader.check("q4_2")

In [ ]:
MIN_BRUTEFORCE_KEYLEN = 2
MAX_BRUTEFORCE_KEYLEN = 5

def get_next_combination(iterable):
    for i in range(MIN_BRUTEFORCE_KEYLEN, MAX_BRUTEFORCE_KEYLEN):
        yield from map("".join, itertools.product(iterable, repeat=i))

def bruteforce_vignere(ciphertext):
    results = []
    
    for combination in get_next_combination(alphabet):
        decrypted = vignere_decrypt(ciphertext, combination)
        newScore = tvd(np_english_frequencies, calculate_proportions(decrypted))
        
        results.append((decrypted, newScore))
    
    return sorted(results, key=lambda x: x[1])[0:5]

**Question 4.3:** Use what you've learned and implemented previously to build the final function -- crack_vignere! This function will take in an arbitrary ciphertext and return the most likely plaintext.

In [ ]:
def crack_vignere(ciphertext):
    keylengths = find_vignere_key_lengths(ciphertext)
    
    finalstrs = []
    
    if len(ciphertext) <= 20:
        return bruteforce_vignere(ciphertext)
    
    for keylen in keylengths:
        ...

In [ ]:
grader.check("q3_1")

In [ ]:
print('...', crack_vignere(vignere_encrypt(sample_text, "samplekey"))[4512:4751], '...')

And with that, you have built a Caesar cipher and Vignere cipher breaker -- something which took hundreds of years to accomplish in the past. Congrats! 

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)